In [1]:
import json

In [3]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/chatbot/blended-skill-talk/blended_skill_talk.json.translate

In [9]:
import re

def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [10]:
with open('blended_skill_talk.json.translate') as fopen:
    data = json.load(fopen)

before, after = [], []

for i in data:
    if len(i) != 2:
        continue
    text = i[1]
    splitted = text.split('[EENND]')
    if len(splitted) != 2:
        continue
    l, r = splitted
    l = cleaning(l)
    r = cleaning(r)
    before.append(l)
    after.append(r)

In [19]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/chatbot/convai2/convai2-0.json.translate
# !wget https://f000.backblazeb2.com/file/malay-dataset/chatbot/convai2/convai2-100000.json.translate

In [11]:
len(before), len(after)

(31877, 31877)

In [22]:
with open('convai2-0.json.translate') as fopen:
    data = json.load(fopen)

for i in data:
    if len(i) != 2:
        continue
    text = i[1]
    splitted = text.split('[EENND]')
    if len(splitted) == 3:
        splitted = splitted[1:]
    if len(splitted) != 2:
        continue
    l, r = splitted
    l = cleaning(l)
    r = cleaning(r)
    before.append(l)
    after.append(r)

In [23]:
with open('convai2-100000.json.translate') as fopen:
    data = json.load(fopen)

for i in data:
    if len(i) != 2:
        continue
    text = i[1]
    splitted = text.split('[EENND]')
    if len(splitted) == 3:
        splitted = splitted[1:]
    if len(splitted) != 2:
        continue
    l, r = splitted
    l = cleaning(l)
    r = cleaning(r)
    before.append(l)
    after.append(r)

In [24]:
len(before), len(after)

(142685, 142685)

In [25]:
list(zip(before[-10:], after[-10:]))

[('Hai apa khabar?', 'Saya hebat pada malam musim gugur ini'),
 ('Adakah itu orang yang anda harapkan.',
  'Ya, dia yang terbaik. Adakah anda fikir begitu?'),
 ('Adakah anda suka sukan? Saya suka tenis dan bermain untuk hidup.',
  'Sukan tidak baik tetapi saya suka pertunjukan memasak dan memasak'),
 ('Pergi ke kedai kopi.',
  'Saya kadang-kadang memainkan lagu-lagu rakyat di kedai kopi. Ia adalah baik.'),
 ('Suami saya seorang doktor jadi saya memasak makanan dan membekukannya.',
  'Itu adalah idea hebat yang saya lajang, jadi saya cukup untuk satu sahaja'),
 ('Nikmati bersendirian, kami membesarkan keluarga kami sehingga masa lapang tidak lama.',
  'Tetapi sukar kerana saya vegan dan rakan-rakan saya tidak'),
 ('Tiga dan satu dalam perjalanan. Anjing itu seperti kanak-kanak juga.',
  'Saya suka anjing mahukan anjing yang serak tetapi belum dapat memilikinya'),
 ('Adakah anda selalu pergi minum kopi.',
  'Setiap pagi sebelum saya pergi ke petugas kebersihan. Ia sukar.'),
 ('Saya suka 

In [30]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/wiki-wizard/informations-0.json.translate
# !wget https://f000.backblazeb2.com/file/malay-dataset/wiki-wizard/informations-100000.json.translate
# !wget https://f000.backblazeb2.com/file/malay-dataset/wiki-wizard/informations-200000.json.translate

In [36]:
from glob import glob

files = glob('informations-*.json.translate')

for file in files:
    print(file)
    with open(file) as fopen:
        data = json.load(fopen)
        
    for i in data:
        if len(i) != 2:
            continue
        splitted = i[1].split('[EENND]')
        if len(splitted) != 3:
            continue
        l, r = splitted[1:]
        l = cleaning(l)
        r = cleaning(r)
        before.append(l)
        after.append(r)

informations-100000.json.translate
informations-200000.json.translate
informations-0.json.translate


In [37]:
list(zip(before[-10:], after[-10:]))

[('saya dibesarkan di nevada.',
  'Nevada adalah yang ke-7 yang paling luas, yang ke-34 paling ramai, tetapi yang ke-9 paling padat penduduknya dari 50 Amerika Syarikat.'),
 ('saya dibesarkan di nevada.',
  'Nevada secara rasmi dikenali sebagai "Negeri Perak" kerana pentingnya perak untuk sejarah dan ekonominya.'),
 ('saya dibesarkan di nevada.',
  'Kawasan di selatan Lembangan Besar terletak di Gurun Mojave, sementara Tasik Tahoe dan Sierra Nevada terletak di pinggir barat.'),
 ('saya dibesarkan di nevada.',
  'Nevada bersempadan dengan Oregon di barat laut, Idaho di timur laut, California di sebelah barat, Arizona di tenggara dan Utah di sebelah timur.'),
 ('nama saya john dan saya tinggal di dallas, tx.',
  'Ia adalah bandar paling padat penduduk di Dallas – Fort Worth metroplex, yang merupakan kawasan metropolitan paling ramai keempat di Amerika Syarikat.'),
 ('saya dibesarkan di nevada.', 'Ibu kota Nevada adalah Carson City.'),
 ('saya dibesarkan di nevada.',
  'Kira-kira 86% tana

In [38]:
import json
import tensorflow as tf
import itertools

filename = 'chatbot.tsv'
with tf.io.gfile.GFile(filename, 'w') as outfile:
        
    for i in range(len(before)):
        outfile.write('%s\t%s\n' % (before[i], after[i]))

In [39]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.t5.model'

In [40]:
def chatbot_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        ['chatbot.tsv']
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def chatbot_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['chatbot: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )


t5.data.TaskRegistry.remove('chatbot_dataset')
t5.data.TaskRegistry.add(
    'chatbot_dataset',
    dataset_fn = chatbot_dataset,
    splits = ['train'],
    text_preprocessor = [chatbot_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)

/home/husein/.local/lib/python3.6/site-packages/t5/models/mesh_transformer.py:210: UserWarning: get_sentencepiece_model_path is deprecated. Please pass the mixture or task vocabulary directly to the Mesh TensorFlow Transformer instead.
  "get_sentencepiece_model_path is deprecated. Please pass the mixture or "


In [41]:
from tqdm import tqdm

nq_task = t5.data.TaskRegistry.get("chatbot_dataset")
ds = nq_task.get_dataset(split='a', sequence_length={"inputs": 1024, "targets": 1024})
results = []
for ex in tqdm(tfds.as_numpy(ds)):
    results.append((ex['inputs'].tolist(), ex['targets'].tolist()))
    
with open('chatbot.tsv.parse', 'w') as fopen:
    json.dump(results, fopen)

325598it [02:29, 2184.75it/s]


In [42]:
results[0]

([15,
  15145,
  5746,
  50,
  709,
  2301,
  30323,
  51,
  189,
  75,
  1773,
  3,
  1791,
  85,
  103,
  1393,
  16727,
  13,
  17,
  353,
  189,
  18,
  192,
  3,
  15,
  3,
  1],
 [1832,
  508,
  6834,
  1481,
  16699,
  6009,
  254,
  18,
  107,
  1393,
  16727,
  13,
  3,
  12485,
  26,
  353,
  15358,
  1097,
  1])

In [43]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load("sp10m.cased.t5.model")

True

In [44]:
sp.DecodeIds(results[0][0])

'chatbot: Saya suka Pizza lebih baik secara peribadi. Kami mempunyai beberapa pizza yang sangat baik di sini. .'

In [45]:
sp.DecodeIds(results[0][1])

'Anda harus mencuba meletakkan hamburger di atas pizza. Rasanya sangat sedap!'